## ACO using Python

In [1]:
import numpy as np; 
import random as rd;
import operator; 
import pandas as pd; 
from matplotlib import pyplot as plt; 
import random as rd;
import collections;
from itertools import permutations

In [2]:
class Graph():
    """
    Graph: class creates distance matrix for TSP problem. 
    """
    size= 0; 
    def __init__(self, rank: int, matrix: list = None):
        """
            :param rank: rank of the cost matrix 
        """
        self.rank = rank
        #self.matrix = np.zeros((rank, rank))
        #self.pheromone = np.asarray([[1 / (rank * rank) for j in range(rank)] for i in range(rank)])
        self.pheromone = [[1/(rank*rank) for j in range(rank)] for i in range(rank)]
        #self.pheromone = np.c_[np.zeros(rank), self.pheromone]
        #self.pheromone = np.r_[np.zeros((1, rank+1)), self.pheromone]
        #self.matrix = np.asarray(matrix); 
        self.matrix = matrix
    def auto_create(self): 
        """
            Auto_create(self): creates random test distance matrix on its own. 
        """
        G= np.zeros((self.rank, self.rank))
        for i in range(self.rank):
            for j in range(self.rank):
                if i==j :
                    continue;
                
                elif(G[i][j]==0):
                    G[i][j]= rd.randint(1, 9)
                    G[j][i]= G[i][j]
                else: 
                    continue;
                
        self.matrix = G.tolist()
        return G
    
    def create(self):
        """
            Create(self): users can create their own customized distance matrix. 
        """
        G= np.zeros((self.rank, self.rank))
        for i in range(self.rank):
            for j in range(self.rank):
                if i==j:
                    continue; 
                    
                elif(G[i][j]==0): 
                    G[j][i] = G[i][j] = int(input(f"cost from city{i} to city{j} "))
                else: 
                    continue; 
        
        self.matrix = G.tolist()
        return G

#help(Graph)
    
    

In [4]:
class Ant(): 
    def __init__(self, graph: Graph, q: int, alpha: float, beta: float, strategy: int): 
        self.graph = graph
        self.Q= q
        self.alpha = alpha
        self.beta = beta
        self.update_strategy = strategy
        self.total_cost = 0
        self.travelled= []
        self.pheromone_delta = [[0 for j in range(graph.rank)] for i in range(graph.rank)]
        #self.pheromone_delta= []
        self.allowed = [i for i in range(graph.rank)]
        self.eta = [[0 if i == j else 1 / graph.matrix[i][j] for j in range(graph.rank)] for i in\
                    range(graph.rank)]
        self.start = rd.randint(0, graph.rank - 1)
        self.travelled.append(self.start)
        self.current = self.start
        self.allowed.remove(self.start)
    
    def _select_next(self):
        denominator = 0
        #print(f"allowed = {self.allowed}")
        #print(f"current= {self.current}")
        for i in self.allowed: 
            denominator += (pow(self.graph.pheromone[i][self.current],self.alpha))\
                            * (pow(self.eta[i][self.current], self.beta))
            
            #print(f"denominator= {denominator}")
            
        probabilites = [0 for i in range(self.graph.rank)]
        for i in range(self.graph.rank):
            try: 
                self.allowed.index(i)
                #print(f"i= {i}")
                probabilites[i]= ((self.graph.pheromone[i][self.current] ** self.alpha)\
                                    * (self.eta[i][self.current] ** self.beta))/ denominator
                
            except ValueError:
                pass

        unvisited = self.allowed
        
        selected = 0
        rand = rd.random()
        for i, probability in enumerate(probabilites):
            rand -= probability
            if rand <= 0:
                selected = i
                break
        
        
        self.allowed.remove(selected)
        self.travelled.append(selected)
        self.total_cost += self.graph.matrix[self.current][selected]
        self.current = selected
        
    def _update_pheromone_delta(self):
        
        for _ in range(1, len(self.travelled)):
            i = self.travelled[_ - 1]
            j = self.travelled[_]
            if self.update_strategy == 1:  # ant-quality system
                self.pheromone_delta[i][j] = self.Q
                
            elif self.update_strategy == 2:  # ant-density system
                self.pheromone_delta[i][j] = self.Q / self.graph.matrix[i][j]
                
            else:  # ant-cycle system
                self.pheromone_delta[i][j] = self.Q / self.total_cost
        
            
        

In [5]:
a= Ant(g, 10, 0.1,0.2, 1)
print(a.pheromone_delta)
print(a.eta)

print(a.allowed)
print(a.start)
#a._select_next()

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]
[[0, 0.16666666666666666, 0.25], [0.16666666666666666, 0, 0.25], [0.25, 0.25, 0]]
[0, 2]
1


In [6]:
class ACO():
    def __init__(self, ant_count: int, generations: int, alpha: float, beta: float, rho: float, q:\
                 int,strategy: int, target: int = None):
    
        """
        :param ant_count:
        :param generations:
        :param alpha: relative importance of pheromone
        :param beta: relative importance of heuristic information
        :param rho: pheromone residual coefficient
        :param q: pheromone intensity
        :param strategy: pheromone update strategy. 0 - ant-cycle, 1 - ant-quality, 2 - ant-density
        :param target: target defines the minimum possible cost in the matrix. 
        """
        
        self.Q = q
        self.rho = rho
        self.beta = beta
        self.alpha = alpha
        self.ant_count = ant_count
        self.generations = generations
        self.update_strategy = strategy
        self.target= target
        #self.graph= graph
        
    def _update_pheromone(self, graph: Graph, ants: list):
        for i, row in enumerate(graph.pheromone):
            for j, col in enumerate(row):
                graph.pheromone[i][j] *= (1-self.rho)
                for ant in ants:
                    graph.pheromone[i][j] += ant.pheromone_delta[i][j]
                    
    def solve(self, graph: Graph):
        """
        graph: graph of cost matrix
        """
        best_cost = float('inf')
        best_solution =  []
        
        for gen in range(self.generations):
            ants = [Ant(graph, self.Q, self.alpha, self.beta, self.update_strategy) for i in range(self.ant_count)]
            
            for ant_no, ant in enumerate(ants): 
                for i in range(graph.rank-1):
                    ant._select_next()
                #print(f"ant-{ant_no+1} travelled path = {ant.travelled} cost= {ant.total_cost + graph.matrix[ant.travelled[-1]][ant.travelled[0]]}")
                
                ant.total_cost += graph.matrix[ant.travelled[-1]][ant.travelled[0]]
                
                if ant.total_cost < best_cost:
                    best_cost = ant.total_cost
                    best_solution =ant.travelled
                
                ant._update_pheromone_delta()
            self._update_pheromone(graph, ants)
            print("<<------------------------------------------------------------------------------>>")
            print(f"generation {gen}: best_cost = {best_cost} & best_solution = {best_solution}")
            
            if self.target!=None: 
                if best_cost <= self.target: 
                    break
            
        return best_solution, best_cost
        
                


In [7]:
g.pheromone

[[0.1111111111111111, 0.1111111111111111, 0.1111111111111111],
 [0.1111111111111111, 0.1111111111111111, 0.1111111111111111],
 [0.1111111111111111, 0.1111111111111111, 0.1111111111111111]]

## Driver function

In [14]:
#ant_count = input('Enter antcount')

graph= [[0, 29, 20, 21, 16, 31, 100, 12, 4, 31, 18],\
                [29, 0, 15, 29, 28, 40, 72, 21, 29, 41, 12],\
                [20, 15, 0, 15, 14, 25, 81, 9, 23, 27, 13],\
                [21, 29, 15, 0, 4, 12, 92, 12, 25, 13, 25],\
                [16, 28, 14, 4, 0, 16, 94, 9, 20, 16, 22],\
                [31, 40, 25, 12, 16, 0, 95, 24, 36, 3, 37],\
                [100, 72, 81, 92, 94, 95, 0, 90, 101, 99, 84],\
                [12, 21, 9, 12, 9, 24, 90, 0, 15, 25, 13],\
                [4, 29, 23, 25, 20, 36, 101, 15, 0, 35, 18],\
                [31, 41, 27, 13, 16, 3, 99, 25, 35, 0 ,38],\
                [18, 12, 13, 25, 22, 37, 84, 13, 18, 38, 0]]

graph= Graph(rank= 11, matrix= graph)

aco = ACO(ant_count= 100, generations= 50, alpha= 1, beta= 2, rho= 0.2, q= 10, strategy = 2, target= 253)
path, cost = aco.solve(graph)

print(f"best path = {path} with cost = {cost}")


<<------------------------------------------------------------------------------>>
generation 0: best_cost = 256 & best_solution = [8, 0, 5, 9, 3, 4, 7, 2, 6, 1, 10]
<<------------------------------------------------------------------------------>>
generation 1: best_cost = 253 & best_solution = [2, 5, 9, 3, 4, 7, 0, 8, 10, 1, 6]
best path = [2, 5, 9, 3, 4, 7, 0, 8, 10, 1, 6] with cost = 253


In [9]:
#benchmark matrix for tsp with target 937

a= """
   0  83  93 129 133 139 151 169 135 114 110  98  99  95  81 152 159 181 172 185 147 157 185 220 127 181
  83   0  40  53  62  64  91 116  93  84  95  98  89  68  67 127 156 175 152 165 160 180 223 268 179 197
  93  40   0  42  42  49  59  81  54  44  58  64  54  31  36  86 117 135 112 125 124 147 193 241 157 161
 129  53  42   0  11  11  46  72  65  70  88 100  89  66  76 102 142 156 127 139 155 180 228 278 197 190
 133  62  42  11   0   9  35  61  55  62  82  95  84  62  74  93 133 146 117 128 148 173 222 272 194 182
 139  64  49  11   9   0  39  65  63  71  90 103  92  71  82 100 141 153 124 135 156 181 230 280 202 190
 151  91  59  46  35  39   0  26  34  52  71  88  77  63  78  66 110 119  88  98 130 156 206 257 188 160
 169 116  81  72  61  65  26   0  37  59  75  92  83  76  91  54  98 103  70  78 122 148 198 250 188 148
 135  93  54  65  55  63  34  37   0  22  39  56  47  40  55  37  78  91  62  74  96 122 172 223 155 128
 114  84  44  70  62  71  52  59  22   0  20  36  26  20  34  43  74  91  68  82  86 111 160 210 136 121
 110  95  58  88  82  90  71  75  39  20   0  18  11  27  32  42  61  80  64  77  68  92 140 190 116 103
  98  98  64 100  95 103  88  92  56  36  18   0  11  34  31  56  63  85  75  87  62  83 129 178 100  99
  99  89  54  89  84  92  77  83  47  26  11  11   0  23  24  53  68  89  74  87  71  93 140 189 111 107
  95  68  31  66  62  71  63  76  40  20  27  34  23   0  15  62  87 106  87 100  93 116 163 212 132 130
  81  67  36  76  74  82  78  91  55  34  32  31  24  15   0  73  92 112  96 109  93 113 158 205 122 130
 152 127  86 102  93 100  66  54  37  43  42  56  53  62  73   0  44  54  26  39  68  94 144 196 139  95
 159 156 117 142 133 141 110  98  78  74  61  63  68  87  92  44   0  22  34  38  30  53 102 154 109  51
 181 175 135 156 146 153 119 103  91  91  80  85  89 106 112  54  22   0  33  29  46  64 107 157 125  51
 172 152 112 127 117 124  88  70  62  68  64  75  74  87  96  26  34  33   0  13  63  87 135 186 141  81
 185 165 125 139 128 135  98  78  74  82  77  87  87 100 109  39  38  29  13   0  68  90 136 186 148  79
 147 160 124 155 148 156 130 122  96  86  68  62  71  93  93  68  30  46  63  68   0  26  77 128  80  37
 157 180 147 180 173 181 156 148 122 111  92  83  93 116 113  94  53  64  87  90  26   0  50 102  65  27
 185 223 193 228 222 230 206 198 172 160 140 129 140 163 158 144 102 107 135 136  77  50   0  51  64  58
 220 268 241 278 272 280 257 250 223 210 190 178 189 212 205 196 154 157 186 186 128 102  51   0  93 107
 127 179 157 197 194 202 188 188 155 136 116 100 111 132 122 139 109 125 141 148  80  65  64  93   0  90
 181 197 161 190 182 190 160 148 128 121 103  99 107 130 130  95  51  51  81  79  37  27  58 107  90   0
 
"""

In [15]:
def createCostMatrix( mat, rank :int):
    arr= mat.split(); 
    arr= np.asarray((arr), dtype= np.int)
    arr= np.reshape(arr, (rank, rank))
    #arr= np.c_[np.zeros((rank), dtype = np.int), arr]
    #arr= np.r_[np.zeros((1, rank+1), dtype= np.int), arr]
    return arr

In [16]:
graph= createCostMatrix(a, rank=26)

graph= Graph(rank= 26, matrix= graph)

aco = ACO(ant_count= 100, generations= 20, alpha= 1, beta= 2, rho= 0.2, q= 1, strategy = 3)
path, cost = aco.solve(graph)

print(f"best path = {path} with cost = {cost}")


<<------------------------------------------------------------------------------>>
generation 0: best_cost = 1302 & best_solution = [24, 20, 16, 15, 18, 19, 17, 11, 21, 25, 22, 23, 7, 6, 8, 9, 10, 12, 14, 13, 2, 5, 3, 4, 1, 0]
<<------------------------------------------------------------------------------>>
generation 1: best_cost = 1059 & best_solution = [4, 5, 3, 6, 10, 12, 11, 9, 8, 7, 15, 18, 19, 17, 16, 20, 21, 25, 23, 22, 24, 0, 14, 13, 2, 1]
<<------------------------------------------------------------------------------>>
generation 2: best_cost = 1059 & best_solution = [4, 5, 3, 6, 10, 12, 11, 9, 8, 7, 15, 18, 19, 17, 16, 20, 21, 25, 23, 22, 24, 0, 14, 13, 2, 1]
<<------------------------------------------------------------------------------>>
generation 3: best_cost = 1002 & best_solution = [25, 21, 20, 16, 17, 19, 18, 15, 11, 12, 10, 9, 8, 7, 6, 3, 4, 5, 1, 2, 13, 14, 0, 24, 22, 23]
<<------------------------------------------------------------------------------>>
generatio

In [218]:
graph= createCostMatrix(a, rank=26)

graph= Graph(rank= 26, matrix= graph)

aco = ACO(ant_count= 100, generations= 20, alpha= 1, beta= 2, rho= 0.2, q= 1, strategy = 3)
path, cost = aco.solve(graph)

print(f"best path = {path} with cost = {cost}")


<<------------------------------------------------------------------------------>>
generation 0: best_cost = 1234 & best_solution = [24, 23, 22, 21, 20, 19, 18, 17, 16, 11, 10, 8, 9, 12, 14, 13, 0, 1, 2, 7, 6, 5, 4, 3, 15, 25]
<<------------------------------------------------------------------------------>>
generation 1: best_cost = 1097 & best_solution = [21, 25, 20, 16, 17, 19, 18, 15, 9, 8, 6, 7, 11, 12, 10, 13, 14, 5, 4, 3, 1, 2, 0, 24, 22, 23]
<<------------------------------------------------------------------------------>>
generation 2: best_cost = 1062 & best_solution = [0, 1, 5, 4, 3, 2, 14, 13, 9, 8, 6, 7, 10, 12, 11, 16, 17, 19, 18, 15, 20, 21, 25, 22, 23, 24]
<<------------------------------------------------------------------------------>>
generation 3: best_cost = 998 & best_solution = [14, 13, 10, 12, 11, 9, 8, 15, 18, 19, 17, 16, 20, 21, 25, 22, 23, 24, 0, 1, 2, 3, 5, 4, 6, 7]
<<------------------------------------------------------------------------------>>
generation

In [219]:
graph= createCostMatrix(a, rank=26)

graph= Graph(rank= 26, matrix= graph)

aco = ACO(ant_count= 100, generations= 20, alpha= 1, beta= 2, rho= 0.2, q= 1, strategy = 3)
path, cost = aco.solve(graph)

print(f"best path = {path} with cost = {cost}")

<<------------------------------------------------------------------------------>>
generation 0: best_cost = 1330 & best_solution = [9, 10, 12, 6, 7, 8, 15, 19, 18, 11, 14, 13, 2, 4, 5, 3, 1, 21, 20, 25, 17, 16, 22, 23, 24, 0]
<<------------------------------------------------------------------------------>>
generation 1: best_cost = 1061 & best_solution = [3, 5, 4, 6, 7, 8, 9, 13, 14, 10, 12, 11, 21, 25, 20, 16, 15, 18, 19, 17, 22, 23, 24, 0, 1, 2]
<<------------------------------------------------------------------------------>>
generation 2: best_cost = 1042 & best_solution = [23, 22, 25, 21, 20, 16, 17, 19, 18, 15, 9, 11, 12, 10, 13, 14, 2, 1, 4, 3, 5, 6, 7, 8, 0, 24]
<<------------------------------------------------------------------------------>>
generation 3: best_cost = 977 & best_solution = [16, 20, 21, 25, 23, 22, 24, 0, 1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 12, 11, 14, 13, 15, 18, 19, 17]
<<------------------------------------------------------------------------------>>
generation

In [220]:
graph= createCostMatrix(a, rank=26)

graph= Graph(rank= 26, matrix= graph)

aco = ACO(ant_count= 100, generations= 20, alpha= 1, beta= 2, rho= 0.2, q= 1, strategy = 3)
path, cost = aco.solve(graph)

print(f"best path = {path} with cost = {cost}")

<<------------------------------------------------------------------------------>>
generation 0: best_cost = 1262 & best_solution = [4, 5, 3, 6, 7, 8, 15, 18, 19, 17, 25, 16, 20, 21, 11, 12, 14, 13, 10, 9, 2, 1, 0, 22, 23, 24]
<<------------------------------------------------------------------------------>>
generation 1: best_cost = 1092 & best_solution = [22, 25, 21, 20, 16, 17, 11, 12, 9, 13, 14, 10, 15, 18, 19, 7, 6, 8, 5, 3, 4, 2, 1, 0, 24, 23]
<<------------------------------------------------------------------------------>>
generation 2: best_cost = 991 & best_solution = [12, 11, 10, 9, 8, 7, 6, 3, 5, 4, 13, 14, 2, 1, 0, 24, 23, 22, 25, 21, 20, 16, 17, 19, 18, 15]
<<------------------------------------------------------------------------------>>
generation 3: best_cost = 991 & best_solution = [12, 11, 10, 9, 8, 7, 6, 3, 5, 4, 13, 14, 2, 1, 0, 24, 23, 22, 25, 21, 20, 16, 17, 19, 18, 15]
<<------------------------------------------------------------------------------>>
generation 

In [222]:
graph= createCostMatrix(a, rank=26)

graph= Graph(rank= 26, matrix= graph)

aco = ACO(ant_count= 100, generations= 20, alpha= 1, beta= 2, rho= 0.2, q= 1, strategy = 3)
path, cost = aco.solve(graph)

print(f"best path = {path} with cost = {cost}")

<<------------------------------------------------------------------------------>>
generation 0: best_cost = 1321 & best_solution = [1, 11, 20, 9, 13, 14, 12, 10, 16, 7, 4, 5, 3, 8, 6, 15, 18, 19, 17, 25, 21, 22, 23, 24, 0, 2]
<<------------------------------------------------------------------------------>>
generation 1: best_cost = 1112 & best_solution = [16, 11, 12, 10, 13, 14, 2, 4, 8, 9, 6, 7, 5, 3, 1, 0, 24, 23, 22, 25, 21, 20, 17, 19, 18, 15]
<<------------------------------------------------------------------------------>>
generation 2: best_cost = 1034 & best_solution = [19, 18, 15, 8, 9, 7, 6, 3, 5, 4, 2, 13, 14, 11, 12, 10, 1, 0, 24, 23, 22, 21, 25, 20, 16, 17]
<<------------------------------------------------------------------------------>>
generation 3: best_cost = 1010 & best_solution = [14, 13, 9, 8, 15, 18, 19, 17, 16, 20, 21, 25, 22, 23, 24, 0, 1, 2, 3, 4, 5, 7, 6, 12, 11, 10]
<<------------------------------------------------------------------------------>>
generatio

In [229]:

graph= [[0, 29, 20, 21, 16, 31, 100, 12, 4, 31, 18],\
                [29, 0, 15, 29, 28, 40, 72, 21, 29, 41, 12],\
                [20, 15, 0, 15, 14, 25, 81, 9, 23, 27, 13],\
                [21, 29, 15, 0, 4, 12, 92, 12, 25, 13, 25],\
                [16, 28, 14, 4, 0, 16, 94, 9, 20, 16, 22],\
                [31, 40, 25, 12, 16, 0, 95, 24, 36, 3, 37],\
                [100, 72, 81, 92, 94, 95, 0, 90, 101, 99, 84],\
                [12, 21, 9, 12, 9, 24, 90, 0, 15, 25, 13],\
                [4, 29, 23, 25, 20, 36, 101, 15, 0, 35, 18],\
                [31, 41, 27, 13, 16, 3, 99, 25, 35, 0 ,38],\
                [18, 12, 13, 25, 22, 37, 84, 13, 18, 38, 0]]

graph= Graph(rank= 11, matrix= graph)

aco = ACO(ant_count= 100, generations= 50, alpha= 1, beta= 2, rho= 0.2, q= 10, strategy = 2, target= 253)
path, cost = aco.solve(graph)

print(f"best path = {path} with cost = {cost}")

<<------------------------------------------------------------------------------>>
generation 0: best_cost = 268 & best_solution = [6, 7, 0, 8, 4, 3, 9, 5, 2, 10, 1]
<<------------------------------------------------------------------------------>>
generation 1: best_cost = 255 & best_solution = [2, 7, 4, 3, 5, 9, 0, 8, 10, 1, 6]
<<------------------------------------------------------------------------------>>
generation 2: best_cost = 255 & best_solution = [2, 7, 4, 3, 5, 9, 0, 8, 10, 1, 6]
<<------------------------------------------------------------------------------>>
generation 3: best_cost = 255 & best_solution = [2, 7, 4, 3, 5, 9, 0, 8, 10, 1, 6]
<<------------------------------------------------------------------------------>>
generation 4: best_cost = 255 & best_solution = [2, 7, 4, 3, 5, 9, 0, 8, 10, 1, 6]
<<------------------------------------------------------------------------------>>
generation 5: best_cost = 255 & best_solution = [2, 7, 4, 3, 5, 9, 0, 8, 10, 1, 6]
<<--

In [232]:
graph= createCostMatrix(a, rank=26)

graph= Graph(rank= 26, matrix= graph)

aco = ACO(ant_count= 100, generations= 20, alpha= 1, beta= 2, rho= 0.2, q= 1, strategy = 3)
path, cost = aco.solve(graph)

print(f"best path = {path} with cost = {cost}")

<<------------------------------------------------------------------------------>>
generation 0: best_cost = 1309 & best_solution = [0, 14, 11, 10, 20, 15, 18, 16, 17, 19, 8, 7, 6, 3, 5, 4, 2, 13, 1, 9, 12, 25, 21, 23, 22, 24]
<<------------------------------------------------------------------------------>>
generation 1: best_cost = 1108 & best_solution = [10, 9, 8, 15, 17, 19, 18, 16, 20, 25, 21, 22, 23, 24, 12, 11, 14, 13, 2, 6, 7, 4, 5, 3, 1, 0]
<<------------------------------------------------------------------------------>>
generation 2: best_cost = 1066 & best_solution = [11, 12, 14, 13, 9, 2, 3, 5, 4, 8, 6, 7, 15, 18, 19, 17, 16, 25, 20, 21, 22, 23, 24, 0, 1, 10]
<<------------------------------------------------------------------------------>>
generation 3: best_cost = 1020 & best_solution = [2, 3, 4, 5, 6, 7, 8, 9, 13, 14, 12, 11, 10, 15, 18, 19, 17, 16, 20, 21, 22, 25, 23, 24, 0, 1]
<<------------------------------------------------------------------------------>>
generatio